#### 20260109 update to intrinsics summaries from calib.db

In [2]:
import json
import pandas as pd
import glob
import os
import numpy as np

#### set up the infrastructure to read calib.db json output

In [15]:
json_dir = r'C:\CJH\python\FRC\vision\2025\python_2025_multicam_2429\intrinsics'
json_dir = r'C:\CJH\python\FRC\vision\2026\intrinsics'

In [16]:
def get_calibration_df(directory_path):
    """
    Scans a directory for .json calibration files and returns a 
    formatted Pandas DataFrame of intrinsic parameters.
    """
    pattern = os.path.join(directory_path, "**", "*.json")
    json_files = glob.glob(pattern, recursive=True)
    
    all_data = []
    for file in json_files:
        with open(file, 'r') as f:
            data = json.load(f)
            
            # Flatten the camera matrix and resolution into top-level keys
            matrix = data['camera_matrix']
            dist = data['distortion_coefficients']
            
            flat_entry = {
                'camera': data.get('camera'),
                'fx': matrix[0][0],
                'fy': matrix[1][1],
                'cx': matrix[0][2],
                'cy': matrix[1][2],
                'd1': dist[0], 'd2': dist[1], 'd3': dist[2], 'd4': dist[3], 'd5': dist[4],
                'x_res': data['img_size'][0],
                'y_res': data['img_size'][1],
                'avg_reprojection_error': data.get('avg_reprojection_error')
            }
            all_data.append(flat_entry)
            
    return pd.DataFrame(all_data)

In [17]:
# this is not averaged 
df = get_calibration_df(json_dir)
df

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB Camera (0c45:6366) RED,898.784549,899.151183,630.395824,369.335779,0.041353,-0.057882,0.000098,0.000361,-0.007250,1280,720,0.212901
1,Arducam OV9281 USB Camera (0c45:6366) RED,904.904326,904.807079,626.580344,369.369034,0.045201,-0.055299,0.000799,-0.000983,-0.011861,1280,720,0.139020
2,Arducam OV9281 USB Camera (0c45:6366) RED,903.690298,903.930989,630.313983,369.038240,0.045356,-0.079693,-0.000016,0.000425,0.018643,1280,720,0.168310


---
####  now average the camera data in case there are outliers

In [18]:
def get_model_averages(df):
    """Returns the mean intrinsic values grouped by camera model."""
    return df.groupby("camera").mean(numeric_only=True).reset_index()

In [19]:
averages = get_model_averages(df)
averages

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB Camera (0c45:6366) RED,902.459724,902.62975,629.096717,369.247685,0.04397,-0.064291,0.000294,-0.000066,-0.000156,1280.0,720.0,0.17341


In [20]:
# Filter for any camera string containing the C920 hardware ID
c920_all = df[df['camera'].str.contains('046d:08e5', case=False)]
c920_all

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error


In [21]:
for _, row in averages.iterrows():
    # Construct the dictionary for unscaled intrinsics
    intrinsics_dict = {
        "fx": round(row['fx'], 2),
        "fy": round(row['fy'], 2),
        "cx": round(row['cx'], 2),
        "cy": round(row['cy'], 2)
    }
    
    # Extract unscaled distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB Camera (0c45:6366) RED :
    "intrinsics": {"fx": 902.46, "fy": 902.63, "cx": 629.1, "cy": 369.25},
    "distortions": [0.04397004, -0.06429124, 0.00029384, -6.56e-05, -0.00015588],


---
#### tools to scale down if we choose to

In [22]:
def scale_intrinsics(target_res, source_data, use_max_focal=True):
    """
    Scales intrinsic parameters from a source resolution to a target resolution.
    
    Args:
        target_res (tuple): (width, height) e.g., (640, 360)
        source_data (dict/Series): Dictionary containing fx, fy, cx, cy, x_res, y_res
        use_max_focal (bool): If True, uses the larger scale factor for focal length 
                              (standard for maintaining FOV during cropping).
    """
    target_x, target_y = target_res
    source_x, source_y = source_data['x_res'], source_data['y_res']
    
    scale_x = target_x / source_x
    scale_y = target_y / source_y
    
    # Focal length scaling
    f_scale = max(scale_x, scale_y) if use_max_focal else (scale_x, scale_y)
    
    if isinstance(f_scale, tuple):
        fx = source_data['fx'] * f_scale[0]
        fy = source_data['fy'] * f_scale[1]
    else:
        fx = source_data['fx'] * f_scale
        fy = source_data['fy'] * f_scale

    # Principal point scaling
    cx = source_data['cx'] * scale_x
    cy = source_data['cy'] * scale_y
    
    # Calculate FOV for verification
    fov_h = 2 * np.degrees(np.arctan(target_x / (2 * fx)))
    fov_v = 2 * np.degrees(np.arctan(target_y / (2 * fy)))
    
    return {
        'fx': round(fx, 3), 'fy': round(fy, 3),
        'cx': round(cx, 2), 'cy': round(cy, 2),
        'fov_h': round(fov_h, 1), 'fov_v': round(fov_v, 1)
    }

In [23]:
# target resolution for the scaled output
target_resolution = (640, 360)

for _, row in averages.iterrows():
    # Scale the average data for this camera
    scaled = scale_intrinsics(target_resolution, row)
    
    # Format the intrinsics dictionary
    intrinsics_dict = {
        "fx": scaled['fx'],
        "fy": scaled['fy'],
        "cx": scaled['cx'],
        "cy": scaled['cy']
    }
    
    # Extract distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} (640x360) :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB Camera (0c45:6366) RED (640x360) :
    "intrinsics": {"fx": 451.23, "fy": 451.315, "cx": 314.55, "cy": 184.62},
    "distortions": [0.04397004, -0.06429124, 0.00029384, -6.56e-05, -0.00015588],


---
#### show min/max on each parameter

In [99]:
# Assuming 'df' is the DataFrame returned by get_calibration_df()
intrinsics_cols = ['fx', 'fy', 'cx', 'cy', 'avg_reprojection_error']

# Group by the camera model and calculate min, max, and mean for each parameter
summary_table = df.groupby('camera')[intrinsics_cols].agg(['min', 'max', 'mean'])

# Optional: Clean up the column names for better readability
summary_table.columns = ['_'.join(col).strip() for col in summary_table.columns.values]

# Display the result
summary_table.reset_index()

,camera,fx_min,fx_max,fx_mean,fy_min,fy_max,fy_mean,cx_min,cx_max,cx_mean,cy_min,cy_max,cy_mean,avg_reprojection_error_min,avg_reprojection_error_max,avg_reprojection_error_mean
0,Arducam OV9281 USB Camera (0c45:6366) A,915.997699,922.677198,919.355796,915.759586,922.960043,919.668696,648.542023,651.026905,649.473605,351.629074,356.870558,354.142917,0.084695,0.117942,0.106459
1,Arducam OV9281 USB Camera (0c45:6366) B,917.609991,922.111195,919.480682,917.789436,922.376487,919.667128,629.409153,634.105079,631.660167,319.204682,323.638127,321.310451,0.119979,0.146790,0.134209
2,Arducam OV9281 USB Camera (0c45:6366) C,903.302048,911.755241,907.477611,903.883499,910.998429,907.244237,639.565341,641.899906,640.898388,351.716775,355.667149,353.932054,0.174544,0.225927,0.193219
3,Arducam OV9281 USB Camera (0c45:6366) D,906.850812,914.496388,912.344847,906.548036,913.775728,911.710152,638.130482,641.891040,640.241789,430.352100,433.823280,431.907270,0.151267,0.212522,0.180646
4,HD Pro Webcam C920 (046d:082d) A,899.578722,943.544533,925.850180,908.384113,942.210821,927.242728,629.636624,659.538920,641.539322,334.364162,379.920347,350.240760,0.173346,0.547980,0.322754
5,HD Pro Webcam C920 (046d:082d) A 202601,898.181799,930.932027,913.515787,899.711831,931.992715,915.986792,632.826604,646.587741,640.281698,341.552318,354.043916,348.040271,0.200359,0.293065,0.240440
6,HD Pro Webcam C920 (046d:08e5) D,946.534460,997.738542,968.355862,949.427351,998.418380,967.505072,609.115581,657.192224,641.909914,347.132442,364.959054,354.769446,0.170899,0.407191,0.297501
7,HD Pro Webcam C920 (046d:08e5) D 202601,934.240514,939.594377,936.917446,934.511796,940.500384,937.506090,637.953667,643.415935,640.684801,354.460820,359.905692,357.183256,0.171084,0.207996,0.189540
8,c922 Pro Stream Webcam (046d:085c) B,950.404937,995.372669,968.285652,949.202323,992.396066,968.173786,639.569603,677.113390,654.426110,323.240883,367.940903,346.705363,0.261106,0.403546,0.336494
9,c922 Pro Stream Webcam (046d:085c) B 202601,939.640912,952.971292,945.646767,940.988463,950.868057,945.874717,638.866572,644.910484,642.947819,348.491346,357.857016,351.470947,0.195423,0.280497,0.225767
